In [1]:
spark.sql("Drop Table icecatalog.mydb.meteorites")

Intitializing Scala interpreter ...

Spark Web UI available at http://bc18817f1332:4045
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1720470742517)
SparkSession available as 'spark'


res0: org.apache.spark.sql.DataFrame = []


## ICEBERG - SCHEMA EVOLUTION
## Objective: To test the schema evolution by adding, updating, deleting columns, and changing the datatype of columns 
## Dataset format: This nested JSON dataset is about meteorite landing which is downloaded from the public source mentioned below
## Dataset Source: https://catalog.data.gov/dataset/

## 1.1 Configure Spark Iceberg Runtime package and other settings

In [2]:
spark.conf.set("spark-sql.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2")
spark.conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
spark.conf.set("spark.sql.catalog.mycatalog", "org.apache.iceberg.spark.SparkSessionCatalog")
spark.conf.set("spark.sql.catalog.mycatalog.type", "Hadoop")

org.apache.spark.sql.AnalysisException:  Cannot modify the value of a static config: spark.sql.extensions.

##packages that need to be downloaded and used during the Spark session
spark.conf.set("spark-sql.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2")

##This specifies any extensions to SQL that should be present in the Spark session
spark.conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

##The settings below are to configure your specific catalog, which can be under a namespace of your choosing eg:here it is spark_catalog
##This specifies that this specific catalog is using the Apache Iceberg Spark Catalog class.
spark.conf.set("spark.sql.catalog.mycatalog", "org.apache.iceberg.spark.SparkSessionCatalog")

##This setting is used to set the type of catalog you are using, and possible values include:Hadoop (if using HDFS/File System Catalog) , Hive
spark.conf.set("spark.sql.catalog.mycatalog.type", "Hadoop")

## 1.2 The spark.sql.extensions might show an error that "Cannot modify the value of a static config: spark.sql.extensions" 
## which means it's not set, However when its value is checked Using getter configuration, it displays the value as set above 
## i.e. IcebergSparkSessionExtensions

In [3]:
spark.conf.get("spark.sql.extensions")

res2: String = org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions


## 2.0 Read the nested json dataset and verify its schema

In [4]:
val jsonDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing.json")

jsonDF: org.apache.spark.sql.DataFrame = [data: array<array<string>>, meta: struct<view: struct<approvals: array<struct<reviewedAt:bigint,reviewedAutomatically:boolean,state:string,submissionDetails:struct<permissionType:string>,submissionId:bigint,submissionObject:string,submissionOutcome:string,submissionOutcomeApplication:struct<failureCount:bigint,status:string>,submittedAt:bigint,submitter:struct<displayName:string,id:string>,targetAudience:string,workflowId:bigint>>, assetType: string ... 38 more fields>>]


In [5]:
jsonDF.printSchema

root
 |-- data: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- meta: struct (nullable = true)
 |    |-- view: struct (nullable = true)
 |    |    |-- approvals: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- reviewedAt: long (nullable = true)
 |    |    |    |    |-- reviewedAutomatically: boolean (nullable = true)
 |    |    |    |    |-- state: string (nullable = true)
 |    |    |    |    |-- submissionDetails: struct (nullable = true)
 |    |    |    |    |    |-- permissionType: string (nullable = true)
 |    |    |    |    |-- submissionId: long (nullable = true)
 |    |    |    |    |-- submissionObject: string (nullable = true)
 |    |    |    |    |-- submissionOutcome: string (nullable = true)
 |    |    |    |    |-- submissionOutcomeApplication: struct (nullable = true)
 |    |    |    |    |    |-- failureCount: long (nullable = true

## source: https://stackoverflow.com/questions/61863489/flatten-nested-json-in-scala-spark-dataframe/61863579#61863579
## 2.1 The dynamic code in scala is referred from the above source where it explodes the nested JSON 
## into individual columns, since all of these are not arrays we cannot use the explode function and also it will make 
## more cumbersome to process individual columns. The below code dynamically splits arrays, structs type into individual columns
## keeping its hierarchy intact, For eg if c is the nested child of b which is a child of an i.e. a.b.c, the code will split it as an a_b_c column 
## This will also prevent duplicating columns in case nested JSON has the same property name because its 
## specific hierarchy will be attached to its name now

In [3]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import scala.annotation.tailrec
import scala.util.Try

implicit class DFHelpers(df: DataFrame) {
    def columns = {
      val dfColumns = df.columns.map(_.toLowerCase)
      df.schema.fields.flatMap { data =>
        data match {
          case column if column.dataType.isInstanceOf[StructType] => {
            column.dataType.asInstanceOf[StructType].fields.map { field =>
              val columnName = column.name
              val fieldName = field.name
              col(s"${columnName}.${fieldName}").as(s"${columnName}_${fieldName}")
            }.toList
          }
          case column => List(col(s"${column.name}"))
        }
      }
    }

    def flatten: DataFrame = {
      val empty = df.schema.filter(_.dataType.isInstanceOf[StructType]).isEmpty
      empty match {
        case false =>
          df.select(columns: _*).flatten
        case _ => df
      }
    }
    def explodeColumns = {
      @tailrec
      def columns(cdf: DataFrame):DataFrame = cdf.schema.fields.filter(_.dataType.typeName == "array") match {
        case c if !c.isEmpty => columns(c.foldLeft(cdf)((dfa,field) => {
          dfa.withColumn(field.name,explode_outer(col(s"${field.name}"))).flatten
        }))
        case _ => cdf
      }
      columns(df.flatten)
    }
}



import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import scala.annotation.tailrec
import scala.util.Try
defined class DFHelpers


## 2.2 Call the above function with the dataframe variable

In [7]:
val flattenedJsonDF = jsonDF.explodeColumns

flattenedJsonDF: org.apache.spark.sql.DataFrame = [data: string, meta_view_approvals_reviewedAt: bigint ... 100 more fields]


## 2.3 Verifying the schema after the function is run. It maintained the hierarchy by separating the 
## names with underscore symbole ('-') between each data column. 
## This will also prevent duplicating columns in case nested json has the same property name because its 
## specific hierarchy will be attached to its name now.
##  For eg: the property 'flags' are now prefixed with its hierarchy meta_view_flags, meta_view_owner_flags, meta_view_columns_flags

In [8]:
flattenedJsonDF.printSchema

root
 |-- data: string (nullable = true)
 |-- meta_view_approvals_reviewedAt: long (nullable = true)
 |-- meta_view_approvals_reviewedAutomatically: boolean (nullable = true)
 |-- meta_view_approvals_state: string (nullable = true)
 |-- meta_view_approvals_submissionDetails_permissionType: string (nullable = true)
 |-- meta_view_approvals_submissionId: long (nullable = true)
 |-- meta_view_approvals_submissionObject: string (nullable = true)
 |-- meta_view_approvals_submissionOutcome: string (nullable = true)
 |-- meta_view_approvals_submissionOutcomeApplication_failureCount: long (nullable = true)
 |-- meta_view_approvals_submissionOutcomeApplication_status: string (nullable = true)
 |-- meta_view_approvals_submittedAt: long (nullable = true)
 |-- meta_view_approvals_submitter_displayName: string (nullable = true)
 |-- meta_view_approvals_submitter_id: string (nullable = true)
 |-- meta_view_approvals_targetAudience: string (nullable = true)
 |-- meta_view_approvals_workflowId: long (

In [9]:
flattenedJsonDF.show(5)

+--------------------+------------------------------+-----------------------------------------+-------------------------+----------------------------------------------------+--------------------------------+------------------------------------+-------------------------------------+-------------------------------------------------------------+-------------------------------------------------------+-------------------------------+-----------------------------------------+--------------------------------+----------------------------------+------------------------------+-------------------+-------------------------+-----------------------+------------------+----------------------------------------------+----------------------------------------+--------------------------------------------+--------------------------------------+----------------------------------------+-----------------------------------------+-------------------------------------+-----------------------------------------+---

## 3. Finally :-), the core step, create an iceberg table and write the json data to it. 
## So now the resultant table is in parquet format, yeyy!!
## Once the catalog is created, for the next writes use append to add to existing table, 
## create for new ,replace to overwrite  or use both createOrReplace for safer side if one wants to replace
## and create new table if it already exists to 

In [10]:
val icedTable = flattenedJsonDF.writeTo("icecatalog.mydb.meteorites").using("iceberg").createOrReplace()

icedTable: Unit = ()


## 3.1 Verify if the table is created using iceberg format

In [11]:
val meteorTable = spark.read.format("iceberg").load("icecatalog.mydb.meteorites")

meteorTable: org.apache.spark.sql.DataFrame = [data: string, meta_view_approvals_reviewedAt: bigint ... 100 more fields]


##3.2 Display result

In [12]:
meteorTable.show(5)

+--------------------+------------------------------+-----------------------------------------+-------------------------+----------------------------------------------------+--------------------------------+------------------------------------+-------------------------------------+-------------------------------------------------------------+-------------------------------------------------------+-------------------------------+-----------------------------------------+--------------------------------+----------------------------------+------------------------------+-------------------+-------------------------+-----------------------+------------------+----------------------------------------------+----------------------------------------+--------------------------------------------+--------------------------------------+----------------------------------------+-----------------------------------------+-------------------------------------+-----------------------------------------+---

##3.3 Check the count to test later when we append the data again

In [13]:
meteorTable.count()

res7: Long = 542500


##3.4 Check the count of columns to test the columsn size post delete step executions

In [14]:
meteorTable.columns.length

res8: Int = 102


## 4 The Schema Evolution Feature test Begins here 
## Not really :-p, its a test before actual test
## Testing if any of the cases of Schema Evolution works by default without further configurations. 
## These 4.x steps are to verify if any of the schema evolution case works by default before configuring schema evolution specific setting
## These executions are run to compare results before and after schema evolution configurations.

## 4.1 From the original dataset, I have deleted and updated(which is as good as adding new column) columns and 
## changed datatype of other column and saved them as new datasets, so that I can write back to the iceberg table 
## where the schema is already set due to the first write execution
## New columns : Added a new string column meta_view_approvals_documentVersion, boolean column documentReviewed and updated(equivalent to new) integer col from meta_view_averageRating to  meta_view_averageDocumentRating
## Deleted column : meta_view_hideFromCatalog
## Steps are - Read the modified JSON dataset, explode it, and then append it to the existing iceberg table

In [15]:
val allchangesColjsonDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing_change.json")
val flattenedallchangesDF = allchangesColjsonDF.explodeColumns
val icedallchangedTable = flattenedallchangesDF.writeTo("icecatalog.mydb.meteorites").append()

org.apache.spark.sql.AnalysisException:  [INSERT_COLUMN_ARITY_MISMATCH.TOO_MANY_DATA_COLUMNS] Cannot write to `demo`.`icecatalog`.`mydb`.`meteorites`, the reason is too many data columns:

## 4.2.1 
## From the original dataset, I have deleted one column 'meta_view_hideFromCatalog' and saved as new dataset,
## a boolean column 'meta_view_hideFromCatalog' with value false and newBackend with value true and an integer column publicationGroup whose value is greater than 0
## so that I can write back to iceberg table where schema is already set due to first write execution
## Steps are - Read the json dataset with one missing column, explode it and then append it to existing iceberg table

In [16]:
val deletedColjsonDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing_deletedmulticols.json")
val flatteneddeletedColJsonDF = deletedColjsonDF.explodeColumns
val icedDeletedColTable = flatteneddeletedColJsonDF.writeTo("icecatalog.mydb.meteorites").append()

org.apache.spark.sql.AnalysisException:  [INCOMPATIBLE_DATA_FOR_TABLE.CANNOT_FIND_DATA] Cannot write incompatible data for the table `demo`.`icecatalog`.`mydb`.`meteorites`: Cannot find data for the output column `meta_view_hideFromCatalog`.

## 4.3 From the original dataset, I have updated one column (which is as good as a new column now) toand saved as new dataset, 
## so that I can write back to iceberg table where schema is already set due to first write execution
## Updated column from meta_view_averageRating to meta_view_averageDocumentRating
## Steps - Read the json dataset with one new column, explode it and then append it to existing iceberg table

In [17]:
val updatedColjsonDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing_UpdatedCol.json")
val flattenedupdatedColJsonDF = updatedColjsonDF.explodeColumns
val icedDeletedColTable = flattenedupdatedColJsonDF.writeTo("icecatalog.mydb.meteorites").append()

org.apache.spark.sql.AnalysisException:  [INCOMPATIBLE_DATA_FOR_TABLE.CANNOT_FIND_DATA] Cannot write incompatible data for the table `demo`.`icecatalog`.`mydb`.`meteorites`: Cannot find data for the output column `meta_view_averageRating`.

#### 4.4 From the original dataset, I have changed the datat type of column  and saved as new dataset, 
## so that I can write back to iceberg table where schema is already set due to first write execution
## changed the data type of column meta_view_createdAt from string to bigint(basically removed the double quotes around the value from json data)
## Read the json dataset with changed data type column, explode it and then append it to existing iceberg table

In [18]:
val deletedColjsonDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing_changeDT.json")
val flatteneddeletedColJsonDF = deletedColjsonDF.explodeColumns
val icedDeletedColTable = flatteneddeletedColJsonDF.writeTo("icecatalog.mydb.meteorites").append()

org.apache.spark.sql.AnalysisException:  [INCOMPATIBLE_DATA_FOR_TABLE.CANNOT_SAFELY_CAST] Cannot write incompatible data for the table `demo`.`icecatalog`.`mydb`.`meteorites`: Cannot safely cast `meta_view_createdAt` "STRING" to "BIGINT".

## 5.0 SCHEMA EVOLUTION TEST(actually) BEGINS HERE 
## Original Source : https://iceberg.apache.org/docs/latest/spark-writes/#schema-merge
## Set the table property to accept any schema as per the above apache documentation source before 

## The documentation from above link states below
## Schema Merge🔗
## While inserting or updating Iceberg is capable of resolving schema mismatch at runtime. 
## If configured, Iceberg will perform an automatic schema evolution as follows:
## A new column is present in the source but not in the target table.
## The new column is added to the target table. Column values are set to NULL in all the rows already present in the table
## A column is present in the target but not in the source.
## The target column value is set to NULL when inserting or left unchanged when updating the row.
## The target table must be configured to accept any schema change by setting the property write.spark.accept-any-schema to true.

In [19]:
spark.sql("ALTER TABLE icecatalog.mydb.meteorites SET TBLPROPERTIES ('write.spark.accept-any-schema'='true')")

res9: org.apache.spark.sql.DataFrame = []


## 5.1 This step is same as 4.1, I am using the dataset where all changes (add, delete, data type change) exist  
## so that I can write back to the iceberg table where the schema is already set due to the first write execution
## New columns : Added a new string column meta_view_approvals_documentVersion, boolean column documentReviewed and updated(equivalent to new) integer col from meta_view_averageRating to  meta_view_averageDocumentRating
## Deleted column : meta_view_hideFromCatalog
## Read the JSON dataset with the changed data type column, explode it, and then append it to the existing iceberg table

In [20]:
val allschemchangesDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing_change.json")
val allschemchangesFlatDF = allschemchangesDF.explodeColumns
val allschemchangesIcedTable = allschemchangesFlatDF.writeTo("icecatalog.mydb.meteorites").option("mergeSchema","true").append()

java.lang.IllegalArgumentException:  Cannot write incompatible dataset to table with schema:

## 5.2.1 This step is similar to 4.2
## From the original dataset, I have deleted three columns of different datatype and saved as new dataset
## a boolean column 'meta_view_hideFromCatalog' with value false and newBackend with value true and an integer column publicationGroup whose value is greater than 0
## so that I can write back to iceberg table where schema is already set due to first write execution
## Read the json dataset with one missing column, explode it and then append it to existing iceberg table

In [1]:
val allschemchangesDelDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing_deletedmulticols.json")

Intitializing Scala interpreter ...

Spark Web UI available at http://bc18817f1332:4045
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1720471709248)
SparkSession available as 'spark'


allschemchangesDelDF: org.apache.spark.sql.DataFrame = [data: array<array<string>>, meta: struct<view: struct<approvals: array<struct<reviewedAt:bigint,reviewedAutomatically:boolean,state:string,submissionDetails:struct<permissionType:string>,submissionId:bigint,submissionObject:string,submissionOutcome:string,submissionOutcomeApplication:struct<failureCount:bigint,status:string>,submittedAt:bigint,submitter:struct<displayName:string,id:string>,targetAudience:string,workflowId:bigint>>, assetType: string ... 35 more fields>>]


In [4]:
val allschemchangesDelFlatDF = allschemchangesDelDF.explodeColumns

allschemchangesDelFlatDF: org.apache.spark.sql.DataFrame = [data: string, meta_view_approvals_reviewedAt: bigint ... 97 more fields]


In [ ]:
val allschemchangesDelIcedTable = allschemchangesDelFlatDF.writeTo("icecatalog.mydb.meteorites").option("mergeSchema","true").append()

## 5.2.2 Since the deletion works or atleast appended without any errors, its worth checking how the table schema looks now.

In [ ]:
val icedmeteorCheckTable = spark.read.format("iceberg").load("icecatalog.mydb.meteorites")
icedmeteorCheckTable.printSchema

## 4.2.3 Verify the count as the data is succesfully appended, it should be exactly twice the earlier result checked at 3.3

In [ ]:
icedmeteorCheckTable.count()

## 4.2.4 Verify the columsn count as one column is deleted now

In [ ]:
icedmeteorCheckTable.columns.length

In [ ]:
icedmeteorCheckTable.select("meta_view_hideFromCatalog","meta_view_newBackend","meta_view_publicationGroup").head(10)

In [ ]:
icedmeteorCheckTable.select("meta_view_hideFromCatalog","meta_view_newBackend","meta_view_publicationGroup").tail(10)

## 5.3 This step is same and using same dataset as 4.3
## From the original dataset, I have updated one column (which is as good as a new column now) toand saved as new dataset, 
## so that I can write back to iceberg table where schema is already set due to first write execution
## Updated column from meta_view_averageRating to meta_view_averageDocumentRating
## Steps - Read the json dataset with one new column, explode it and then append it to existing iceberg table

In [5]:
val allschemchangesUpDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing_UpdatedCol.json")
val allschemchangesUpFlatDF = allschemchangesUpDF.explodeColumns
val allschemchangesUpIcedTable = allschemchangesUpFlatDF.writeTo("icecatalog.mydb.meteorites").option("mergeSchema","true").append()

java.lang.IllegalArgumentException:  Cannot write incompatible dataset to table with schema:

## 5.4.1 This step is same as 4.4 
## From the original dataset, I have changed the datat type of column  and saved as new dataset, 
## so that I can write back to iceberg table where schema is already set due to first write execution
##changed the data type of column meta_view_createdAt from string to bigint(basically removed the double quotes around teh value from json data)
## Read the json dataset with changed data type column, explode it and then append it to existing iceberg table

In [6]:
val allschemchangesTypeDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing_changeDT.json")
val allschemchangesTypeFlatDF = allschemchangesTypeDF.explodeColumns
val allschemchangesTypeIcedTable = allschemchangesTypeFlatDF.writeTo("icecatalog.mydb.meteorites").option("mergeSchema","true").append()

java.lang.IllegalArgumentException:  Cannot change column type: meta_view_createdAt: long -> string

## 5.4.2 This step is similar to 4.4 however here, I have downcasted the column datatype to check if downcasting works
## From the original dataset, I have changed the datat type of column  and saved as new dataset, 
## so that I can write back to iceberg table where schema is already set due to first write execution
##changed the data type of column meta_view_averageRating from LONG to DOUBLE
## Read the json dataset with changed data type column, explode it and then append it to existing iceberg table

In [7]:
val allschemchangesDownTypeDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing-changeDTup.json")
val allschemchangesDownTypeFlatDF = allschemchangesDownTypeDF.explodeColumns
val allschemchangesDownTypeIcedTable = allschemchangesDownTypeFlatDF.writeTo("icecatalog.mydb.meteorites").option("mergeSchema","true").append()

java.lang.IllegalArgumentException:  Cannot change column type: meta_view_averageRating: long -> double